# <font color=green>1. MEU PRIMEIRO SCRAPING

# 1. Ambiente e bibliotecas

In [14]:
import time
import re

import bs4
import urllib.request as urllib_request
import pandas

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pandas.__version__)

BeautifulSoup -> 4.11.1
urllib -> 3.8
pandas -> 1.5.2


In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

### Convertando o tipo bytes para string

In [3]:
def str_to_valid_str(str_data):
    return " ".join(str_data.split()).replace("> <", "><")

In [4]:
def bytes_to_valid_str(b_data):
    str_data = b_data.decode("utf-8")
    return str_to_valid_str(str_data)

In [5]:
def get_html(page):
    html = str_to_valid_str(page)
    
    soup = BeautifulSoup(html, "html.parser")
    return soup.prettify()

In [6]:
def click_button(btn):
    driver.execute_script("arguments[0].click();", btn)

In [7]:
def close_popup(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Close":
                print("    Found button")
                close_btn = btn
        if close_btn != None:
            click_button(close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Popup closed ", e.reason)

In [8]:
def change_to_table_view(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Table View":
                close_btn = btn
        click_button(close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Change to table view ", e.reason)

In [9]:
def scroll_page(value):
    element_list = driver.find_elements(
        By.CSS_SELECTOR, "div[data-test-id='virtuoso-scroller']"
    )
    driver.execute_script("arguments[0].scroll(0," + str(value) + ");", element_list[0])

# 2. Meu primeiro scraping

In [ ]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

path = "https://mindapps.org"
## path = "https://boardgamegeek.com/" ## Test path
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}

try:
    req = Request(path, headers = headers)
    response = urlopen(req)
    html = response.read()
    
    html = bytes_to_valid_str(html)
    
    soup = BeautifulSoup(html, "html.parser")
    print(soup.prettify())
    
except HTTpError as e:
    print(e.status, e.reason)

except URLError as e:
    print(e.reason)

# Selenium 

In [ ]:
options = Options()
# options.headless = True
options.add_argument("--window-size=1920,820")

DRIVER_PATH = '/home/pizzaisamor/chromedriver/chromedriver'
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

driver.get('https://mindapps.org/Apps')

driver.set_window_position(0, 0)
driver.set_window_size(1920, 820)

try:
    element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "app-content"))
    )
    
    driver.set_window_size(1920, 820)
    
    # print(get_html(driver.page_source))
    #close_popup(driver)
    
    time.sleep(15)
    
    # change_to_table_view(driver)
    
    #table = driver.find_element(
    #    By.XPATH, '//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div'
    #)
    
    # Find first card
    navigate = driver.find_element(
        By.XPATH, '//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div/div/div[1]/div'
    )
    
    click_button(navigate) # Click on the card
    
    time.sleep(5) # Load page
    # Get info about number of comments
    coments_info = driver.find_element(
        By.XPATH, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[4]'
    )

    time.sleep(10) # Load info
    quant = re.split('\)|\(', coments_info.text)[1] # Get coments quantity
    print(quant)
    
    # Get cost line
    cost = driver.find_element(
        By.XPATH, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[3]'
    )
    print(cost.text)
    
    driver.back()
    
    #for i in range(16):
    #    element = driver.find_element(
    #        By.XPATH, 
    #        '//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div/div[' 
    #        + str(i + 1) 
    #        + ']'
    #    )
    
    #    print(element.text, "\n\n")
    
    time.sleep(1000)
    
finally:
    driver.quit()

/tmp/ipykernel_13629/3344811575.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


6
Cost:
Free to Download
Subscription
In-App Purchase


---
# <font color=green>3. INTRODUÇÃO AO BEAUTIFULSOUP

# 3.2. Criando um objeto BeautifulSoup

## https://www.crummy.com/software/BeautifulSoup/

### Sobre parser ver: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#parser-installation

In [ ]:
<div class="ReactVirtualized__Grid__innerScrollContainer" role="rowgroup" style="width: 1855px; height: 18864px; max-width: 1855px; max-height: 18864px; overflow: hidden; position: relative;"><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 0px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/o9trIu4yGHdKMSnCNm9XLWucQBJNc_k1wtwskN-dldNzMI1YFZ97a50TvXPr4OR30L4" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">&lrm;MyPossibleSelf: Mental Health</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by My Possible Self Ltd</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 0px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon Nov 14th 2022 2:00 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 48px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/m8C1o9X_5g2sSU8YNb5SL7jMqRMfPX3YisZe8lN0WLZ8pd5qKIXDk4EGA_0_CmTFKOI" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">zSober AA, AA Meetings,Rehab</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Kenyon Avenue LLC</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 48px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Fri Dec 9th 2022 5:56 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 96px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/zooKtA1eYO7PEwhcvO7TudUKFhmcQMUGQpsjDoEtAiHvddlr9xedlvMarCWNw688_3M" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Zemedy: The IBS Care Program</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Bold Health</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 96px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon Sep 12th 2022 10:53 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 144px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/_0v_dDm_VqcAPfFbWlga7HhFftNVr2D-SIPaULwbbgi4OT-gAOHoa6Nw6EbFUmHtDI0" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Youper: Self-Guided Therapy</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Youper, Inc</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 144px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon Nov 14th 2022 1:17 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 192px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/nmq0ycufXByhHXh6TTYPfnDwkWp9I6yNaVSmW_d5hEax_VbrzwluJQi0dI4lCiJcn60n" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">You Can Quit Smoking</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by CityApps Digital Services</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 192px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue July 19th 2022 11:56 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 240px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/_2t27Nwv0sJgq3XsUCP-haxf0C17xzIEVtO4Pfrp_cSJG-VJAWzIC0DAlVF2kEx8ol2M" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Wysa: Anxiety, therapy chatbot</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Touchkin</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 240px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon Aug 22nd 2022 10:31 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 288px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://is4-ssl.mzstatic.com/image/thumb/Purple116/v4/31/59/e0/3159e08e-8894-c3f7-aca1-663e12808372/AppIcon-1x_U007emarketing-0-10-0-85-220.png/512x512bb.jpg" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Worry Watch: Anxiety &amp; Mood</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Akilan Rajendran</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 288px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon July 11th 2022 4:52 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 336px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/3Rz4MJY4cyH5B3h1ql0JGapQlP9GKgbvWIP10g1mBAJliywDM8C41wbKByD_H9rq2A" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Worry Knot</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by IntelliCare</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 336px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue Nov 15th 2022 6:37 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 384px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/rjrDgxHyBABwzGXfckxdNPhRgXxCkG6AXeTxhH86ZaYv3iw3Qzn69luxlQ_s-a9wuwc" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Womaze: Self-Care &amp; Motivation</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Womaze App</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 384px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue Nov 15th 2022 6:16 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 432px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/a3y33ApEs0aQW5xOn4NwO2Rw1WKP9j3A3dIrDwtHnIPsRCaCfS7973zFYFkSN7TwKLk" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Woebot: Your Self-Care Expert</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Woebot Health Inc</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 432px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue Oct 25th 2022 5:52 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 480px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/dCyVtayN9PXA-h3f03sWIWMWxt25IZ7nG6yWSog-oioc78nO4QKC4P-q7vowNY6ZrO4" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Wisdo: Mental Health &amp; Support</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Wisdo LTD.</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 480px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Sat Oct 15th 2022 4:09 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 528px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/EuBjyvSmvSa_1mizO8MhqX1fgqvnfxLbCx1iB4C4zYpW23t0wYet6dJWxUiWIRKO6w4" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">White Noise Sleep Sounds: rain, nature, and more</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Ipnos Software</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 528px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Wed Aug 24th 2022 10:51 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 576px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/Jb9-HxJx0KSHSfXE8xQ5m3L7c65KCU5_xz5He4fsPmRlpgGoaK5u_HHrju_fzQVDlQ" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">White Noise Pro - ASMR</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by coolsharp</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 576px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue Oct 4th 2022 4:16 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 624px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/lt74xc_uXdwtWGp3ySv128QZ5__ixh67DNrkr8H5om11JAHl13W5A--wcooBztJA3HqH" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">What's Up? - Mental Health App</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Jackson Tempra</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 624px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Thu July 28th 2022 11:41 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 672px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/XCKsIir8qEzk8JVV0Ayl1eu7wrO67fQ6ZBGHBjiFLwpuLnLn_WPkDV1mUXz-0qrmxg" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Welzen: meditations to relax, focus &amp; sleep better</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Welzen LLC.</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 672px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Mon Aug 8th 2022 2:10 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 720px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/qH-LGI_OtoDpIScIoGCadzLVqPjq_PXGKinK0wfBL1a5L_962XcLF6xkQYHZqqtHFYU" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">WellTrack - Interactive Self-Help Therapy</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by ProtoCall Services</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 720px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Thu July 21st 2022 3:12 PM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 768px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://is3-ssl.mzstatic.com/image/thumb/Purple114/v4/97/46/3b/97463b2e-44e3-7d2e-eb95-e11f50b704c6/AppIcon-0-0-1x_U007emarketing-0-0-0-7-0-0-sRGB-0-0-0-GLES2_U002c0-512MB-85-220-0-0.png/512x512bb.jpg" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">Well - Feel Better</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by Luni</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 768px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Thu Dec 1st 2022 12:50 AM</p></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 0px; position: absolute; top: 816px; width: 330px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><ul class="MuiList-root MuiList-dense css-1uzmcsd"><li class="MuiListItem-root MuiListItem-dense MuiListItem-padding css-1vqoiez"><div class="MuiListItemAvatar-root css-a5kqs7" style="min-width: 48px;"><img src="https://play-lh.googleusercontent.com/G5Bdkmoe-7cfnmjGCYN_bvykyBSLyL9rrhETgh7JLEdKPWQGf7ZhnSjpnDNFI7EpaVk" alt="logo" style="height: 40px;"></div><div class="MuiListItemText-root MuiListItemText-dense MuiListItemText-multiline css-1xar93x"><span class="MuiTypography-root MuiTypography-body2 MuiListItemText-primary css-f00wsl">WEconnect Recovery</span><p class="MuiTypography-root MuiTypography-body2 MuiListItemText-secondary css-14bis4s"> by WEconnect</p></div></li></ul></span><span style="float: right;"></span></div></div><div class="MuiTableCell-root MuiTableCell-sizeSmall jss118 jss115 css-i7bgym" style="height: 48px; left: 330px; position: absolute; top: 816px; width: 225px; padding-left: 16px; padding-right: 0px;"><div class="jss123"><span style="flex: 1 1 auto;"><p class="MuiTypography-root MuiTypography-body2 css-ldokli">Tue Aug 30th 2022 12:57 AM</p></span><span style="float: right;"></span></div></div></div>

# 3.3. Acessando tags

In [ ]:
//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div
//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div/div[1]
//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]
//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div[1]/div[2]/div/div[2]/div/div/div[16]

//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div/div/div[1]/div
//*[@id="app-content"]/div[2]/div[3]/div[2]/div/div/div[1]/div/div

//*[@id="app-content"]/div[2]/div/div[4]/div/div[4]

//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[3] ## cost
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[4] ## privacy
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[5] ## clinical_foundation
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[6] ## features
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[7] ## condition_supported
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[8] ## engagements
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[9] ## inputs
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[10] ## outputs
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[11] ## uses
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[12] ## developer_types
//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[13] ##access

//*[@id="app-content"]/div[2]/div/div[4]/div/div[6]/div/nav/ul/li[1] # <
//*[@id="app-content"]/div[2]/div/div[4]/div/div[6]/div/nav/ul/li[2] # 1
//*[@id="app-content"]/div[2]/div/div[4]/div/div[6]/div/nav/ul/li[6] # 5

# 3.4. Acessando o conteúdo das tags

# 3.5. Acessando os atributos de uma tag

---
# <font color=green>4. PESQUISANDO COM O BEAUTIFULSOUP

# 4.1. Os métodos *find()* e *findAll()*

- ### *find(tag, attributes, recursive, text, **kwargs)*

- ### *findAll(tag, attributes, recursive, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all

> **Observação:**
> - *findAll()* também pode ser utilizado como *find_all()*

### Método *find()*

### Método *findAll()*

### Comando equivalente ao método *find()*

### Atalho para o método *findAll()*

### Passando listas de TAGs

### Utilizando o argumento *attributes*

### Buscando por conteúdo de uma TAG

### Utilizando diretamente os atributos

### Cuidado com o atributo "class"

### Obtendo todo o conteúdo de texto de uma página

# 4.2. Outros métodos de pesquisa

- ### *findParent(tag, attributes, text, **kwargs)*

- ### *findParents(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-parents-and-find-parent

> **Observação:**
> - *findParent()* e *findParents()* também podem ser utilizados como *find_parent()* e *find_parents()*, respectivamente.
---
- ### *findNextSibling(tag, attributes, text, **kwargs)*

- ### *findNextSiblings(tag, attributes, text, limit, **kwargs)*

- ### *findPreviousSibling(tag, attributes, text, **kwargs)*

- ### *findPreviousSiblings(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-next-siblings-and-find-next-sibling
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-previous-siblings-and-find-previous-sibling

> **Observação:**
> - *findNextSibling()*, *findNextSiblings()*, *findPreviousSibling()* e *findPreviousSiblings()* também podem ser utilizados como *find_next_sibling()*, *find_next_siblings()*, *find_previous_sibling()* e *find_previous_siblings()*, respectivamente.
---
- ### *findNext(tag, attributes, text, **kwargs)*

- ### *findAllNext(tag, attributes, text, limit, **kwargs)*

- ### *findPrevious(tag, attributes, text, **kwargs)*

- ### *findAllPrevious(tag, attributes, text, limit, **kwargs)*

#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-next-and-find-next
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all-previous-and-find-previous

> **Observação:**
> - *findNext()*, *findAllNext()*, *findPrevious* e *findAllPrevious* também podem ser utilizados como *find_next()*, *find_all_next()*, *find_previous()* e *find_all_previous()*, respectivamente.

## HTML de exemplo para ilustrar a utilização dos métodos de pesquisa do BeautifulSoup

<img src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/BeautifulSoup-method.png" width=80%>

---
## Resultado

### Tratamentos para a string HTML

### Criando o objeto BeautifulSoup

### Parents

## Siblings

## Next e Previous

# <font color=green>5. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE UM ANÚNCIO

# 5.1. Identificando e selecionando os dados no HTML

### Obtendo o HTML e criando o objeto BeautifulSoup

### Criando variávels para armazenar informações

### Obtendo os dados do primeiro CARD

# 5.2. Obtendo o VALOR do veículo anunciado

### <font color=red>Resumo

In [ ]:
# Valor


# 5.3. Obtendo as INFORMAÇÕES sobre o veículo anunciado

### <font color=red>Resumo

In [ ]:
# Informações


# 5.4. Obtendo os ACESSÓRIOS do veículo anunciado

### <font color=red>Resumo

In [ ]:
# Acessórios


# 5.5 Criando um DataFrame com os dados coletados do Alura Motors

# 5.6. Obtendo a FOTO do anúncio

### Visualizando a FOTO no notebook (extra)

### Rotina para acessar e salvar a FOTO do anúncio

## https://docs.python.org/3/library/urllib.request.html#urllib.request.urlretrieve

### <font color=red>Resumo

# <font color=green>6. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE TODOS OS ANÚNCIOS DE UMA PÁGINA

# 6.1. Identificando as informações no HTML

# 6.2. Criando uma rotina de scraping

# <font color=green>7. WEB SCRAPING DO SITE ALURA MOTORS - OBTENDO OS DADOS DE TODOS OS ANÚNCIOS DO SITE

# 7.1. Identificando as informações no HTML

# 7.2. Criando uma rotina de scraping